<center><h1>Chapter 6 Connection</h1></center>

In [1]:
import numpy as np
import pandas as pd

## 1. Relational connection
### 1. Basic concept of connection

It is a common operation to connect two related tables according to a key or a group of keys. For example, the score tables of each subject of the student's final exam are connected according to $\color{red}{name}$ and $\color{red}{class}$ to form a total score table. For example, various information tables of enterprise employees are connected and summarized according to $\color{red}{employee ID number}$. It can be seen that in relational connections, $\color{red}{key}$ is very important and is often represented by the `on` parameter.

Another important factor is the form of the connection. In the relational connection functions `merge` and `join` in `pandas`, the `how` parameter is provided to represent the connection form, which is divided into left connection `left`, right connection `right`, inner connection `inner`, and outer connection `outer`. Their differences can be represented by the following diagram:

<img src="../source/_static/ch6_1.png" width="50%">

As can be seen from the figure, the so-called left connection is based on the key of the left table. If the key in the right table exists in the left table, it is added to the left table, otherwise it is treated as a missing value. The right connection is handled similarly. The inner connection is only responsible for merging the keys that appear on both sides at the same time, while the outer connection will include the keys that appear only on the left and only on the right based on the inner connection.The value that appears on the right, so the outer join is also called a full join.

In the simple example above, there is no duplication of keys in the same table, so how should duplicate keys be handled? You only need to grasp one principle, that is, as long as the values ​​appear on both sides at the same time, they are added in the form of Cartesian product, and if they appear on one side, they are processed according to the join form. Among them, the Cartesian product can be explained by the following example: Suppose the key `Zhang San` in the left table appears twice, and `Zhang San` in the right table also appears twice, then match them one by one, and the final table must contain `2*2` rows with the name `Zhang San`. The following is a schematic diagram of the corresponding example:

<img src="../source/_static/ch6_2.png" width="60%">

Obviously, different join forms should be used in different occasions. Among them, left join and right join are equivalent. Since the keys in their results are determined by the table on one side, they are often used to add to the target table in a direction. The tables on both sides of the inner and outer join are often in similar positions (swapping the positions of the left and right tables does not change the results). If you want to extract the intersection or union of the keys, the specific operation needs to be determined according to the business needs.

### 2. Value connection

In the example in the above diagram, the two tables are connected based on the value of a column. In fact, they can also be connected by a combination of several columns. This value-based connection can be implemented by the `merge` function in `pandas`, for exampleLeft join of a graph:

In [2]:
df1 = pd.DataFrame({'Name':['San Zhang','Si Li'], 'Age':[20,30]})
df2 = pd.DataFrame({'Name':['Si Li','Wu Wang'], 'Gender':['F','M']})
df1.merge(df2, on='Name', how='left')

,Name,Age,Gender
0,San Zhang,20,NaN
1,Si Li,30,F


If the columns you want to connect in the two tables do not have the same column name, you can specify it using `left_on` and `right_on`:

In [3]:
df1 = pd.DataFrame({'df1_name':['San Zhang','Si Li'], 'Age':[20,30]})
df2 = pd.DataFrame({'df2_name':['Si Li','Wu Wang'], 'Gender':['F','M']})
df1.merge(df2, left_on='df1_name', right_on='df2_name', how='left')

,df1_name,Age,df2_name,Gender
0,San Zhang,20,NaN,NaN
1,Si Li,30,Si Li,F


If there are duplicate column names in the two tables, you can specify them using the `suffixes` parameter. For example, when merging test scores, the first table records the Chinese scores and the second table records the math scores:

In [4]:
df1 = pd.DataFrame({'Name':['San Zhang'],'Grade':[70]})
df2 = pd.DataFrame({'Name':['San Zhang'],'Grade':[80]})
df1.merge(df2, on='Name', how='left', suffixes=['_Chinese','_Math'])

,Name,Grade_Chinese,Grade_Math
0,San Zhang,70,80


Sometimes duplicate elements are troublesome, for example, two students are from different classes, but have the same name. In this case, you need to specify the `on` parameter to multiple columns to connect correctly:

In [5]:
df1 = pd.DataFrame({'Name':['San Zhang', 'San Zhang'],
                    'Age':[20, 21],
                    'Class':['one', 'two']})
df2 = pd.DataFrame({'Name':['San Zhang', 'San Zhang'],
                    'Gender':['F', 'M'],
                    'Class':['two', 'one']})
df1

,Name,Age,Class
0,San Zhang,20,one
1,San Zhang,21,two


In [6]:
df2

,Name,Gender,Class
0,San Zhang,F,two
1,San Zhang,M,one


In [7]:
df1.merge(df2, on='Name', how='left') # 错误的结果

,Name,Age,Class_x,Gender,Class_y
0,San Zhang,20,one,F,two
1,San Zhang,20,one,M,one
2,San Zhang,21,two,F,two
3,San Zhang,21,two,M,one


In [8]:
df1.merge(df2, on=['Name', 'Class'], how='left') # 正确的结果

,Name,Age,Class,Gender
0,San Zhang,20,one,M
1,San Zhang,21,two,F


From the above example, if the key is not unique, there will be problems in the connection based on uniqueness. The number of rows in the example is small, but if there are hundreds of thousands to millions of rows in the actual data to be merged, if you want to ensure uniqueness, in addition to using `duplicated` to check whether there are duplicates, `merge` also provides a `validate` parameter to check the uniqueness mode of the connection. There are three modes here, namely one-to-one connection `1:1`, one-to-many connection `1:m`, and many-to-one connection `m:1`. The first one means that the keys of the left and right tables are unique, and the latter two refer to the uniqueness of the left table key and the uniqueness of the right table key respectively.

#### [Practice]
In the above example with multiple columns as keys, the wrong writing is obviously a many-to-many connection, while the correct writing is a one-to-one connection. Please modify the original table so that the correct writing with multiple columns as keys can pass the `validate='1:m'` test, but cannot pass the `validate='m:1'` test.
#### 【END】

### 3. Index join

The so-called index join is to use the index as the key, so it is essentially the same as value join. In `pandas`, the `join` function is used to handle index joins. Its parameter selection is less than that of `merge`. In addition to the required `on` and `how`, the left and right suffixes `lsuffix` and `rsuffix` can be specified for repeated columns. Among them, `oThe n` parameter refers to the index name. When the parameter is omitted for single-layer indexing, the connection is made according to the current index.

In [9]:
df1 = pd.DataFrame({'Age':[20,30]}, index=pd.Series(['San Zhang','Si Li'],name='Name'))
df2 = pd.DataFrame({'Gender':['F','M']}, index=pd.Series(['Si Li','Wu Wang'],name='Name'))
df1.join(df2, how='left')

,Age,Gender
Name,,
San Zhang,20,NaN
Si Li,30,F


Following the example in Section 2, write a `join` version that combines language and math scores:

In [10]:
df1 = pd.DataFrame({'Grade':[70]}, index=pd.Series(['San Zhang'], name='Name'))
df2 = pd.DataFrame({'Grade':[80]}, index=pd.Series(['San Zhang'], name='Name'))
df1.join(df2, how='left', lsuffix='_Chinese', rsuffix='_Math')

,Grade_Chinese,Grade_Math
Name,,
San Zhang,70,80


If you want to perform an operation similar to merge with multiple columns as keys, join needs to use multi-level indexes. For example, the last example in merge can be written as follows:

In [11]:
df1 = pd.DataFrame({'Age':[20,21]}, index=pd.MultiIndex.from_arrays([['San Zhang', 'San Zhang'],['one', 'two']], names=('Name','Class')))
df2 = pd.DataFrame({'Gender':['F', 'M']}, index=pd.MultiIndex.from_arrays([['San Zhang', 'San Zhang'],['two', 'one']], names=('Name','Class')))
df1

Age
Name      Class     
San Zhang one     20
          two     21

In [12]:
df2

Gender
Name      Class       
San Zhang two        F
          one        M

In [13]:
df1.join(df2)

Age Gender
Name      Class            
San Zhang one     20      M
          two     21      F

## 2. Directional connection
### 1. concat

Relational connection was introduced earlier, and the most important parameters are `on` and `how`. However, sometimes users do not care which column is used as the key to merge, but just want to splice two or more tables vertically or horizontally. For this demand, `pandas` provides the `concat` function to achieve it.

In `concat`, the three most commonly used parameters are `axis, join, keys`, which respectively indicate the splicing direction, the connection form, and the name of the old table from which the new table comes. It should be noted here that `join` and `keys` have nothing to do with the concept of `join` function and key mentioned earlier.

In the default state, `axis=0` means splicing multiple tables vertically, which is often used for splicing multiple samples; while `axis=1` means splicing multiple tables horizontally, which is often used for splicing multiple fields or features.

For example, vertically merge the information of people in each table:

In [14]:
df1 = pd.DataFrame({'Name':['San Zhang','Si Li'], 'Age':[20,30]})
df2 = pd.DataFrame({'Name':['Wu Wang'], 'Age':[40]})
pd.concat([df1, df2])

,Name,Age
0,San Zhang,20
1,Si Li,30
0,Wu Wang,40


Merge fields from each table horizontally:

In [15]:
df2 = pd.DataFrame({'Grade':[80, 90]})
df3 = pd.DataFrame({'Gender':['M', 'F']})
pd.concat([df1, df2, df3], 1)

,Name,Age,Grade,Gender
0,San Zhang,20,80,M
1,Si Li,30,90,F


Although `concat` is a function that handles relational merging, it still connects based on indexes. Vertical concatenation will align columns based on column indices. By default, `join=outer` means that all columns are retained and non-existent values ​​are set to missing; `join=inner` means that columns that appear in both tables are retained. Horizontal concatenation will align columns based on row indices, and the `join` parameter can be set similarly.

In [16]:
df2 = pd.DataFrame({'Name':['Wu Wang'], 'Gender':['M']})
pd.concat([df1, df2])

,Name,Age,Gender
0,San Zhang,20.0,NaN
1,Si Li,30.0,NaN
0,Wu Wang,NaN,M


In [17]:
df2 = pd.DataFrame({'Grade':[80, 90]}, index=[1, 2])
pd.concat([df1, df2], 1)

,Name,Age,Grade
0,San Zhang,20.0,NaN
1,Si Li,30.0,80.0
2,NaN,NaN,90.0


In [18]:
pd.concat([df1, df2], axis=1, join='inner')

,Name,Age,Grade
1,Si Li,30,80


Therefore, when you confirm that you want to use a direct directional merge of multiple tables, especially a horizontal merge, you can first use the `reset_index` method to restore the default integer index before merging to prevent erroneous results caused by misaligned indexes and Cartesian products of duplicate indexes.

Finally, the use scenario of the `keys` parameter is that after multiple tables are merged, the user still wants to know which original table the data in the new table comes from. In this case, you can use the `keys` parameter to generate a multi-level index for marking. For example, the first table contains students from Class 1, and the second table contains students from Class 2. You can merge them in the following way:

In [19]:
df1 = pd.DataFrame({'Name':['San Zhang','Si Li'], 'Age':[20,21]})
df2 = pd.DataFrame({'Name':['Wu Wang'],'Age':[21]})
pd.concat([df1, df2], keys=['one', 'two'])

Name  Age
one 0  San Zhang   20
    1      Si Li   21
two 0    Wu Wang   21

### 2. Merge sequence and table

Use `concat` to realize the directional splicing between multiple tables. If you want to append a sequence to the end of the row or column of the table, you can use `append` and `assign` methods respectively.

In `append`, if the original table is the index of the default integer sequence, you can use `ignore_index=True` to automatically label the index corresponding to the new sequence, otherwise you must specify the `name` attribute for `Series`.

In [20]:
s = pd.Series(['Wu Wang', 21], index = df1.columns)
df1.append(s, ignore_index=True)

,Name,Age
0,San Zhang,20
1,Si Li,21
2,Wu Wang,21


For `assign`, although it can be used to add new columns, it is generally equivalent to adding new columns through the form of `df['new_col'] = ...`. At the same time, the disadvantage of using `[]` to modify is that it will directly modify the original table, while `assign` returns a temporary copy:

In [21]:
s = pd.Series([80, 90])
df1.assign(Grade=s)

,Name,Age,Grade
0,San Zhang,20,80
1,Si Li,21,90


In [22]:
df1['Grade'] = s
df1

,Name,Age,Grade
0,San Zhang,20,80
1,Si Li,21,90


## 3. Quasi-join operations

In addition to the several join functions introduced above, `pandas` also designs some functions that can perform certain operations on two tables, which are collectively referred to as quasi-join operations here.

### 1. Compare

`compare` is a new function introduced after `1.1.0`, which can compare the differences between two tables or sequences and summarize them:

In [23]:
df1 = pd.DataFrame({'Name':['San Zhang', 'Si Li', 'Wu Wang'],
                    'Age':[20, 21 ,21],
                    'Class':['one', 'two', 'three']})
df2 = pd.DataFrame({'Name':['San Zhang', 'Li Si', 'Wu Wang'],
                    'Age':[20, 21 ,21],
                    'Class':['one', 'two', 'Three']})
df1.compare(df2)

Name         Class       
    self  other   self  other
1  Si Li  Li Si    NaN    NaN
2    NaN    NaN  three  Three

The result returns the rows and columns where the different values ​​are located. If they are the same, they will be filled with missing values ​​`NaN`, where `other` and `self` refer to the passed parameter table and the called table itself respectively.

If you want to fully display the comparison of all elements in the table, you can set `keep_shape=True`:

In [24]:
df1.compare(df2, keep_shape=True)

Name         Age        Class       
    self  other self other   self  other
0    NaN    NaN  NaN   NaN    NaN    NaN
1  Si Li  Li Si  NaN   NaN    NaN    NaN
2    NaN    NaN  NaN   NaN  three  Three

### 2. Combination

The `combine` function allows two tables to be combined according to certain rules, and automatically aligns column indexes when comparing rules. For the function passed in, the input parameter in each operation is the same-named `Series` from the two tables, and the columns passed in in sequence are the union of the column names of the two tables. For example, the following example will pass in four groups of sequences, `A, B, C, D`, each group is two sequences from the left and right tables. At the same time, when comparing the `A` column, `s2` refers to a completely empty sequence because it does not exist in the called table, and the sequence index from the first table will be `reindex` into the union of the two indexes. The specific process can be viewed by inserting the appropriate `print` method in the passed in function.

The following example selects the element with the smaller corresponding index position:

In [25]:
def choose_min(s1, s2):
    s2 = s2.reindex_like(s1)
    res = s1.where(s1<s2, s2)
    res = res.mask(s1.isna()) # isna表示是否为缺失值，返回布尔序列
    return res
df1 = pd.DataFrame({'A':[1,2], 'B':[3,4], 'C':[5,6]})
df2 = pd.DataFrame({'B':[5,6], 'C':[7,8], 'D':[9,10]}, index=[1,2])
df1.combine(df2, choose_min)

,A,B,C,D
0,NaN,NaN,NaN,NaN
1,NaN,4.0,6.0,NaN
2,NaN,NaN,NaN,NaN


#### 【Practice】
Please modify the above code to keep the original values ​​of the 4 corresponding positions in `df2` that are not replaced by `df1`.
#### 【END】
In addition, setting the `overwrite` parameter to `False` can keep the columns in $\color{red}{called table}$ that do not appear in the passed parameter table without setting the non-missing values:

In [26]:
df1.combine(df2, choose_min, overwrite=False)

,A,B,C,D
0,1.0,NaN,NaN,NaN
1,2.0,4.0,6.0,NaN
2,NaN,NaN,NaN,NaN


#### 【Practice】
In addition to `combine`, `pandas` also has a `combine_first` method, which is used to fill in the values ​​in the first table if the values ​​in the second table at the corresponding index position in the first table are not missing when combining two tables. Here is an example. Please use the `combine` function to complete the same function.
#### 【END】

In [27]:
df1 = pd.DataFrame({'A':[1,2], 'B':[3,np.nan]})
df2 = pd.DataFrame({'A':[5,6], 'B':[7,8]}, index=[1,2])
df1.combine_first(df2)

,A,B
0,1.0,3.0
1,2.0,7.0
2,6.0,8.0


## 4. Exercises
### Ex1: US epidemic data set

There are epidemic reports from April 12 to November 16 in the United States (in the `/data/us_report` folder). Please merge `Confirmed, Deaths, Recovered, Active` of `New York` into one table, with the index being a sequence of date strings generated as follows:

In [28]:
date = pd.date_range('20200412', '20201116').to_series()
date = date.dt.month.astype('string').str.zfill(2) +'-'+ date.dt.day.astype('string').str.zfill(2) +'-'+ '2020'
date = date.tolist()
date[:5]

['04-12-2020', '04-13-2020', '04-14-2020', '04-15-2020', '04-16-2020']

### Ex2: Implement the join function

Please implement the `join` function with the `how` parameter

* Assume that the two tables to be joined have no common columns
* The calling method is `join(df1, df2, how="left")`
* Give a test example